# [Learning to learn by gradient descent by gradient descent](https://arxiv.org/abs/1606.04474)

*by deepmind on NIPS 2016*
- SEOHASONG - Korea Univ.
- 2018.03.25
- Deep Paper Study 신촌

## Abstract

사람들은 목적달성을 위해 경영, 공학, 정치 등 다양한 분야에서 결과에 영향을 주는 변수를 찾고자 노력합니다.

사람이 직접 변수를 찾아낼 수도 있지만, 기계학습이 발전하면서
컴퓨터가 자동으로 변수를 찾는 방식도 주목받고 있습니다.

기계학습에서 알고리즘은 최적화 알고리즘입니다.
최적화알고리즘은 예측이 틀린 정도가 최소가 되도록 설계됩니다.

컴퓨터는 데이터에 알고리즘을 적용해 자동으로 적절한 변수를 찾는 역할을 하며,
여기 사용되는 알고리즘은 사람이 직접 만들어 줘야 합니다.

이 논문에서는 사람이 직접 만들던 최적화 알고리즘을 학습하는 문제를 논의합니다.

- 기존에는 사람이 만든 최적화 알고리즘을 통해 변수를 학습했습니다.
- 앞으로는 최적화 알고리즘을 변수로 두고 그것이 무엇일지 자동으로 학습하고자 합니다.

이 논문의 목적은 *특정 분류의 문제에 대해 잘 동작하는 알고리즘*을 만드는 일련의 과정을 개발하는 것입니다.

*알고리즘 만들기*를 우리가 풀어야 하는 문제로 생각하는 것을 **Meta learning**이라고 부릅니다.

Meta learning 관련 논문으로는 AutoML등이 있으며,

Meta learning은 Few-shot learning(One-shot, Zero-shot 등등)과도 관련이 있습니다.

## Introduction

일반적으로 기계학습 문제는 목적함수라고 불리는

### $f(\theta),\quad\theta\in\Theta$

를 최적화 하는 문제입니다.

식 $f(\theta)$을 최소화(최대화) 하는 파라미터 $\theta \in \Theta$를 찾는 것이 목표입니다.

### $\theta^* = arg\ min_{\theta\in\Theta}\ f(\theta)$

식 $f$는 보통 파라미터 $\theta$에 대해 미분 가능한 함수이며,

이경우 일반적으로 단계적으로 업데이트가 이루어지는 Gradient descent 방식을 이용합니다.

### $\theta_{t+1} = \theta_t\ -\ \alpha_t\nabla f(\theta_t)$

Gradient descent방식은 치명적인 약점이 있습니다.

***단계적으로 한번 미분한 값만 고려하고 두번 미분한 값은 고려하지 못합니다.***

Gradient descent가 아닌 전통적인 최적화 기법에서는 이런 문제를 피할 방법이 있습니다.

함수의 두번 미분한 정도(곡률)의 정보를 이용하는 것입니다.

> 참조
> - Hessian matrix of second-order partial derivatives
> - Generalized Gauss-Newton matrix
> - Fisher information matrix
> - ...

현대의 많은 최적화 기법은 업데이트 규칙을 특정 문제에 깔맞춤하는 방식으로 이루어 졌습니다.

예를들어, 요즘 유행하고 있는 딥러닝 분야에서는

- 고차원 백터공간에서
- Convex가 아닌 형태에서

효과적인 업데이트 규칙을 이용합니다.

> 참조
> - momentum
> - Rprop
> - Adagrad
> - RMSprop
> - ADAM
> - ...

```
More focused methods can also be applied when more structure of the optimization problem is known [Martens and Grosse, 2015]. In contrast, communities who focus on sparsity tend to favor very different approaches [Donoho, 2006, Bach et al., 2012]. This is even more the case for combinatorial optimization for which relaxations are often the norm [Nemhauser and Wolsey, 1988].
```

현실의 각 산업군에서는 해당 분야에서 잘 동작하는 최적화 방식을 연구하지만,

각 방식들은 다른 분야에서는 제대로 동작하지 않을 수도 있습니다.

```
Moreover the No Free Lunch Theorems for Op- timization [Wolpert and Macready, 1997] show that in the setting of combinatorial optimization, no algorithm is able to do better than a random strategy in expectation.
```

이러한 연구들이 시사하는 바는, 일반적으로

1. 분야를 세분화 하고
2. 세분화된 분야에 특화된 방식을 찾는 것

이 유일한 성능 향상의 접근법일수도 있다는 것입니다.

이 논문은 업데이트 규칙을 사람이 직접 만드는 방식에서 학습되는 방식으로 다르게 구하고자 합니다.

학습되는 optimizer(최적화 방식)를 여기서는 $g$라고 명명하겠습니다.

우리는 $g$를 파라미터의 집합 $\phi$에 깔맞춤 되도록 할 것입니다.

기존의 gradient방식은

### $\theta_{t+1} = \theta_t\ -\ \alpha_t\nabla f(\theta_t)$

다음과 같이 수정됩니다.

### $\theta_{t+1} = \theta_t\ -\ g_t(\nabla f(\theta_t), \phi)$

- 여기서 $g$는 meta-learner라고 부르기도 합니다.
- 여기서 $f$는 learner라고 부르기도 합니다.

![img](img/0.png)

> 왼쪽의 optimizer가 오른쪽의 optimizee 성능에 대한 정보를 받고 optimizee를 수정합니다.

### 1.1 Transfer learning and generalization

이 논문에서 풀고하 하는 문제는 Meta learning의 관점에서 논의됩니다.

Meta learning의 관점에서 generalization은 평소와는 약간 다른 의미를 가집니다.

일반적인 기계학습에서 generalization은

알고리즘이 학습된 적 없는 데이타에 대해서 어떻게 동작하는지에 관심이 있습니다.

이 논문에서는

1. 학습된 적 없는 데이터들이
2. 알고리즘에서 사용할 데이터입니다.
3. 그리고 이경우 generalization은
4. transfer learning이 어떻게 되는지에 관심이 있습니다.
5. ex) MNIST to CIFAR10

#### Transfer Learning

기존의 만들어진 모델을 사용하여 새로운 모델을 만들시 학습을 빠르게 하며, 예측을 더 높이는 방법입니다.

실질적으로 Convolution network을 처음부터 학습시키는 일은 많지 않습니다. 대부분의 문제는 이미 학습된 모델을 사용해서 문제를 해결할 수 있습니다.

복잡한 모델일수록 학습시키기 어렵습니다. 어떤 모델은 2주정도 걸릴수 있으며, 비싼 GPU 여러대를 사용하기도 합니다.

layers의 갯수, activation, hyper parameters등등 고려해야 할 사항들이 많으며, 실질적으로 처음부터 학습시키려면 많은 시도가 필요합니다.

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.

1. 새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우
> 데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다. 새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.

2. 새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우
> 새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.

3. 새로 훈련할 데이터가 적으며 original 데이터와 다른 경우
> 데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.

4. 새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우
> 데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

### 1.2 A brief history and related work

```python
def a_brief_history_and_related_work():
    pass
```

## 2 Learning to learn with recurrent neural networks

optimizer의 파라미터 $\phi$를 통해 최종 optimizee의 파라미터 $\theta$를 얻게 될 것입니다.

최종 optimizee의 파라미터들은 $\theta^*(f, \phi))$로 표기합니다.

우리는 이 파라미터 $\theta^*$가 적용된 $f$들을 구하고,

$f$들의 평균을 $\phi$를 이용한 optimizer의 비용으로 생각 할 수 있습니다.

> "$f$들"은 정확히 무엇인가요..? 

### $L(\phi)=E_f\left[ f(\theta^*(f, \phi)) \right] $

### - $\theta^*(f, \phi))$: final optimizee parameters
### - $\phi$: optimizer parameters
### - $f$: function in question

우리는 비용에 대해 일련의 업데이트 과정을 거쳐가야합니다.

이 일련의 업데이트는 RNN구조가 적합해 보입니다. RNN 알고리즘을 $m$이라고 부르도록 하겠습니다.

$m$의 아웃풋이 바로 $t$시점의 업데이트에 적용될 $g_t$가 됩니다.

RNN 알고리즘 $m$은 파라피터 $\phi$를 가지며, $h_t$를 입력으로 받아 상태가 결정되는 함수입니다.

### $L(\phi)=E_f\left[ \sum _{ t=1 }^{ T }{ w_tf(\theta_t) } \right]$
### $\qquad$ where
### $\qquad\qquad \theta_{t+1}=\theta_t+g_t$
### $\qquad\qquad \left[ \begin{matrix} g_t \\ h_{t+1} \end{matrix} \right] = m(\nabla_t, h_t, \phi)$


비용함수가 위와 같이 바뀌었습니다.

$\nabla_t=\nabla_{\theta}f(\theta_t)$와 같이 간단히 표기합니다.

위 식에서 $t=T$일 경우만 $w_t=1$이며, 나머지경우는 $w_t=0$일 경우 처음 제시한 식과 같아집니다.

이제 다 왔습니다.

다음의 계산그래프를 이용해 $L(\phi)$를 optimizer의 파라미터 $\phi$를 학습해서 최소화하면 됩니다.

![img](img/1.png)

optimizer는 $\partial L(\phi)/\partial\phi$를 계산해 업데이트 됩니다.

이 과정에서 만약 최종 $\nabla_T$만을 사용한다면,

RNN을 학습 시킬 때 효과적으로 gradient descent가 이루어지지 않습니다.

따라서 모든$t$ 단계에 대해서 $f_t$를 구하고 이를 사용합니다.

논문에서는 간단하게 모든 $w_t$를 1로 두었습니다.

계산그래프에서는 실선을 통해서만 backpropagation을 수행합니다. (점선의 weight를 drop합니다.)

이는 optimizee의 파라미터가 optimizer의 파라미터 $\phi$에 영향 받지 않는다는 가정을 의미합니다.

### $\partial \nabla_t/\partial\phi = 0$

> 이는 $f$의 2계 도함수를 구할 필요가 없다는 것과 같은 말이랍니다..



### 2.1 Coordinatewise LSTM optimizer

이 과정을 수행하기 위해서는, 적어도 10개의 수천개의 파라미터를 가진 알고리즘을 계산해야합니다.

따라서, fully connected RNN을 이용하기에는 비용이 너무 큽니다.

따라서 coordinatewise network architecture를 이용합니다.

![img](img/2.png)

사실 LSTM은 한개이며 LSTM의 파라미터를 공유합니다.

optimizee의 각 파라미터들에 대해서, 공통된 optimizer의 파라미터를 이용합니다.

($\theta^n$이 하나의 배치와도 같다고 생각하기)

## 3 Experiments

![img](img/3.png)
![img](img/4.png)
![img](img/5.png)
![img](img/6.png)

참고자료
- [PR-031](https://www.youtube.com/watch?v=p55H46RiZ6k&t=0s&list=PLlMkM4tgfjnJhhd4wn5aj8fVTYJwIpWkS&index=32)
- [incredible.ai](http://incredible.ai/artificial-intelligence/2017/05/13/Transfer-Learning/)
- [deepmind github](https://github.com/deepmind/learning-to-learn)

In [8]:
%%html

<style>
img[alt='img']{
    display: inline-block;
    max-width: 800px;
}
</style>